In [20]:
from flask import Flask, request, jsonify
from threading import Thread
import json

from src.agents.meeting_agent import meeting_agent
from src.agents.priority_agent import get_priority_async
from src.agents.slots_agent import slots_agents

from src.utils.attendees import get_attendees_with_from
from src.utils.interval_handling import get_free_time_slots,get_free_time_slots_priority

In [21]:
import time
import asyncio
from datetime import datetime, timedelta
from dateutil import parser

# Alternative version with even more optimization
async def your_meeting_assistant_ultra_optimized(data): 
    # Start timer for all concurrent operations
    start_total = time.time()

    # Start timer
    start = time.time()
    
    # Create tasks for all independent operations
    priority_task = get_priority_async(data["EmailContent"])
    meeting_agent_task = meeting_agent.run(data["EmailContent"])

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"meeting_agent Runtime: {runtime:.4f} seconds")
    

    # Start timer
    start = time.time()
    
    # Get attendees synchronously (assuming this is fast)
    attendees = get_attendees_with_from(data)
    print("Extracted Attendees")
    print(attendees)

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"get_attendees_with_from Runtime: {runtime:.4f} seconds")
    
    # Wait for priority and meeting agent results
    priority_result, meeting_agent_results = await asyncio.gather(
        priority_task,
        meeting_agent_task
    )
    
    # Process results
    priority = priority_result.data.priority
    data["Priority"] = priority
    
    response_data_meeting = {
        "start_timeframe": meeting_agent_results.data.start_timeframe,
        "end_timeframe": meeting_agent_results.data.end_timeframe,
        "duration_minutes": meeting_agent_results.data.duration_minutes,
        "success": True,
        "message": "Successfully extracted meeting timeframes"
    }
    
    print("EXTRACTED TIME ELEMENTS")
    print(response_data_meeting)
    
    # Get free slots
    free_slots, user_schedule_dict = get_free_time_slots(
        attendees, 
        response_data_meeting["duration_minutes"], 
        response_data_meeting["start_timeframe"], 
        response_data_meeting["end_timeframe"],
        slots_agents
    )
    
    print("Extracted Free Slots")
    if free_slots == 'No available time slots found for the specified duration\n ':
        print('No available time slots found for the specified duration\n ')
        free_slots = get_free_time_slots_priority(
            attendees, 
            response_data_meeting["duration_minutes"], 
            response_data_meeting["start_timeframe"], 
            response_data_meeting["end_timeframe"], 
            priority, 
            len(data["Attendees"])
        )
    print(free_slots)

    # Get slots suggestion
    slots_agent_results = await slots_agents.run(
        f"Free Slots {free_slots} Duration: {response_data_meeting['duration_minutes']}"
    )
    
    response_data_slots = {
        "start_time_of_meeting": slots_agent_results.data.start_time_of_meeting,
        "end_time_of_meeting": slots_agent_results.data.end_time_of_meeting,
        "duration_minutes": slots_agent_results.data.duration_minutes,
        "metadata": slots_agent_results.data.metadata
    }
    
    print("Suggested Slot")
    print(response_data_slots)
    
    print("After Processing\n")
    # Parse the start time string to datetime object
    start_time_dt = dateutil.parser.isoparse(slots_agent_results.data.start_time_of_meeting)
    
    # Add the duration
    end_time_dt = start_time_dt + timedelta(minutes=int(slots_agent_results.data.duration_minutes))
    
    # Convert back to ISO format string
    end_time_str = end_time_dt.isoformat()

    response_data_slots['end_time_of_meeting'] = str(end_time_str)

    print(response_data_slots)

    # Parse the response
    raw = response_data_slots["start_time_of_meeting"].strip()
    try:
        dt = parser.isoparse(raw)
    except Exception:
        dt = parser.parse(raw) 
    
    available_start_time = dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    available_end_dt = dt + timedelta(minutes=response_data_meeting["duration_minutes"])
    available_end_time = available_end_dt.strftime("%Y-%m-%dT%H:%M:%S%z")

    # End total timer
    end_total = time.time()
    total_runtime = end_total - start_total
    print(f"Total optimized runtime: {total_runtime:.4f} seconds")

    # Build output efficiently
    output = {
        "Request_id": data["Request_id"],
        "Datetime": data["Datetime"],
        "Location": data["Location"],
        "From": data["From"],
        "Attendees": [
            {**attendee, "events": user_schedule_dict[attendee["email"]]} 
            for attendee in data["Attendees"]
        ],
        "Subject": data["Subject"],
        "EmailContent": data["EmailContent"],
        "EventStart": available_start_time,
        "EventEnd": available_end_time,
        "Duration_mins": str(response_data_meeting["duration_minutes"]),
        "MetaData": response_data_slots['metadata']
    }
    
    return output

In [24]:
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033c5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Project Status",
    "EmailContent": "Hi Team. Let's meet on Tuesday at 11:00 A.M and discuss about our on-going Projects."
}
start = time.now()
await your_meeting_assistant_ultra_optimized(data)
end = time.now()
runtime = end - start
print(f"{runtime:.4f}")

AttributeError: module 'time' has no attribute 'now'

In [4]:
from flask import Flask, request, jsonify
import asyncio
import json

app = Flask(__name__)
received_data = []

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    
    # Run async function in sync context
    new_data = asyncio.run(your_meeting_assistant_ultra_optimized(data))

    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
